kig på data

In [1]:
import pandas as pd

In [ ]:
path = "data/diagnosis.csv"
diagnosis_df = pd.read_csv(path, sep=';')

df = pd.read_parquet('data/CaseRigshospitalet_optimized.parquet')

 # Remove all Akut contacts
df = df[df['Indlæggelsesmåde'] != 'Akut']

In [16]:
df = df[df['Indlæggelsesmåde'] != 'Akut']

df = df[df["Patient ID"].astype(str).str.strip().ne("")]

df = df[df["Aktionsdiagnosekode"].astype(str).str.strip().ne("")]

### Diagnosis tokens

In [18]:


code = "Aktionsdiagnosekode"
text = "Aktionsdiagnosetekst"
token_dict = {}

for idx, row in diagnosis_df.iterrows():
    code_val = row["Aktionsdiagnosekode"]
    text_val = row["Aktionsdiagnosetekst"]

    text1 = str(code_val) if pd.notnull(code_val) else ""
    text2 = str(text_val) if pd.notnull(text_val) else ""
    combined_text = text1 + " " + text2
    token_dict[code_val] = combined_text

    
   

In [20]:
unique_diag_lists = df.groupby("Patient ID")["Aktionsdiagnosekode"].unique()

patient_records = {}

for patient_id, codes in unique_diag_lists.items():
    diagnoses = [token_dict[str(code)] for code in codes if str(code) in token_dict]
    patient_records[patient_id] = {
        "diagnoses": diagnoses
        # you can add 'medications' and 'context' here if you have them
    }


C:\Users\Jacob\AppData\Local\Temp\ipykernel_12272\820759188.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  unique_diag_lists = df.groupby("Patient ID")["Aktionsdiagnosekode"].unique()


### Demographics token

we need age_group, gender, civil_tilstand and kommune

In [26]:
# make age groups

age_bins = [0, 5, 18, 25, 35, 45, 55, 65, 75, 80, 85, 90, 99,  float("inf")]
age_labels = [f"Age_Group_{str(i).zfill(2)}" for i in range(13)]
df["Age_Group"] = pd.cut(
    df["Patient alder på kontaktstart tidspunkt"],
    bins=age_bins,
    labels=age_labels,
    right=True,
    include_lowest=True
)

In [ ]:

df["Age_Group"] = pd.cut(
    df["Patient alder på kontaktstart tidspunkt"],
    bins=age_bins,
    labels=age_labels,
    include_lowest=True,
    right=True
)

# Clean up relevant columns
df["Patient køn"] = df["Patient køn"].astype(str).str.strip()
df["Patient civilstand"] = df["Patient civilstand"].astype(str).str.strip()
df["Patient kommune"] = df["Patient kommune"].astype(str).str.strip()
df["Age_Group"] = df["Age_Group"].astype(str).str.strip()

# Build demographic tokens (context) per row
def demographic_tokens(row):
    tokens = []
    if row["Patient køn"]:
        tokens.append(f"Sex_{row['Patient køn']}")
    if row["Patient civilstand"]:
        tokens.append(row["Patient civilstand"])
    if row["Patient kommune"]:
        tokens.append(f"Kommune_{row['Patient kommune']}")
    if row["Age_Group"]:
        tokens.append(row["Age_Group"])
    return tokens

df["context"] = df.apply(demographic_tokens, axis=1)

# Remove rows with no context tokens
df = df[df["context"].map(lambda x: len(x) > 0)]

# Group by patient and take the first valid context list per patient
context_dict = (
    df.groupby("Patient ID")["context"]
    .first()
    .to_dict()
)


### Pyro

In [ ]:
import pyro
import pyro.distributions as dist
import torch
from pyro.nn import PyroSample
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoDelta

def model(w_diag, w_context, num_topics, vocab_diag_size, vocab_context_size):
    num_records = len(w_diag)

    alpha = 0.1 * torch.ones(num_topics)
    beta_diag = 0.1 * torch.ones(vocab_diag_size)
    beta_context = 0.1 * torch.ones(vocab_context_size)

    # Topic-word distributions for diagnoses and context
    phi_diag = pyro.sample("phi_diag", dist.Dirichlet(beta_diag).expand([num_topics]).to_event(1))  # (T x V_diag)
    phi_context = pyro.sample("phi_context", dist.Dirichlet(beta_context).expand([num_topics]).to_event(1))  # (T x V_context)

    for v in pyro.plate("records", num_records):
        # Topic mixture for this record
        theta_v = pyro.sample(f"theta_{v}", dist.Dirichlet(alpha))

        # Diagnoses
        for n in pyro.plate(f"diag_tokens_{v}", len(w_diag[v])):
            z = pyro.sample(f"z_diag_{v}_{n}", dist.Categorical(theta_v))
            pyro.sample(f"w_diag_{v}_{n}", dist.Categorical(phi_diag[z]), obs=w_diag[v][n])

        # Contextual tokens
        for n in pyro.plate(f"context_tokens_{v}", len(w_context[v])):
            z = pyro.sample(f"z_context_{v}_{n}", dist.Categorical(theta_v))
            pyro.sample(f"w_context_{v}_{n}", dist.Categorical(phi_context[z]), obs=w_context[v][n])


In [2]:

import pyro
import pyro.distributions as dist
import torch
from pyro.nn import PyroSample
from pyro.infer import SVI, Trace_ELBO
from pyro.infer.autoguide import AutoDelta

def model(w_diag, w_context, time_spent, num_topics, vocab_diag_size, vocab_context_size):
    num_records = len(w_diag)

    alpha = 0.1 * torch.ones(num_topics)
    beta_diag = 0.1 * torch.ones(vocab_diag_size)
    beta_context = 0.1 * torch.ones(vocab_context_size)

    # Per-topic word distributions
    phi_diag = pyro.sample("phi_diag", dist.Dirichlet(beta_diag).expand([num_topics]).to_event(1))
    phi_context = pyro.sample("phi_context", dist.Dirichlet(beta_context).expand([num_topics]).to_event(1))

    # Regression parameters for hospital time
    with pyro.plate("topic_coeffs"):
        beta = pyro.sample("beta", dist.Normal(0., 1.).expand([num_topics]).to_event(1))
    sigma = pyro.sample("sigma", dist.HalfCauchy(1.0))

    for v in pyro.plate("records", num_records):
        # Latent topic mixture for the record
        theta_v = pyro.sample(f"theta_{v}", dist.Dirichlet(alpha))

        for n in pyro.plate(f"diag_tokens_{v}", len(w_diag[v])):
            z = pyro.sample(f"z_diag_{v}_{n}", dist.Categorical(theta_v))
            pyro.sample(f"w_diag_{v}_{n}", dist.Categorical(phi_diag[z]), obs=w_diag[v][n])

        for n in pyro.plate(f"context_tokens_{v}", len(w_context[v])):
            z = pyro.sample(f"z_context_{v}_{n}", dist.Categorical(theta_v))
            pyro.sample(f"w_context_{v}_{n}", dist.Categorical(phi_context[z]), obs=w_context[v][n])

        # Time spent modeled as a linear regression on theta
        mu = (theta_v * beta).sum()
        pyro.sample(f"time_spent_{v}", dist.Normal(mu, sigma), obs=time_spent[v])

